**Importing Pre-requisite Libraries**

In [32]:
#Importing google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
#Importing pre-requisite libaries
import os
import re
import nltk
import glob
import tqdm
import pickle
import random
import string
import inflect
import operator
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk import tokenize
from nltk import word_tokenize
import matplotlib.pyplot as plt
from collections import Counter
from nltk.corpus import stopwords
from prettytable import PrettyTable
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [34]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [35]:
#Fetching stopwords
stopWords = set(stopwords.words('english'))

**Pre-processing the data and defining Utility methods**

In [36]:
#Creating function to delete the special characters
def removeSpecialCharactersFunction(dataText): 
    targetRE = r'[^a-zA-Z0-9\s]'
    textAfterRemovingSpecialChar = re.sub(targetRE,'',dataText)
    return textAfterRemovingSpecialChar

In [37]:
def find_unique(words): 
    unique_words = list(set(words))
    word_freq = {}
    for word in unique_words:
        word_freq[word] = words.count(word)
    return word_freq

In [38]:
#Creating function to find unique words and its frequency in the document given as input
def uniqueWordsFinderFunction(textWords): 
    uniqueWordsList = list(set(textWords))
    wordFrequencyDict = {}
    for word in uniqueWordsList:
        wordFrequencyDict[word] = textWords.count(word)
    return wordFrequencyDict

In [39]:
#Creating function to perform pre-processing steps
def preprocessingData(dataPath):
    fileDetails = {}
    uniqueWordsDict = {}    
    idOfDocument = 1 
    fileData=[]
    print(color.BOLD+"File Id"," ","File Name"+color.END)
    for singleFile in glob.glob(dataPath): 
        filePath = singleFile
        fileName = singleFile.split("/")[-1]        
        fileName = fileName.split(".")[0]                                           
        if os.path.isdir(singleFile):     
            print(fileName)                      
        else:
            print(idOfDocument," \t ",fileName)
            singleFile = open(singleFile,"r",encoding='ISO-8859-1')        
            document = singleFile.read()                                            # Reading file data

            document = removeSpecialCharactersFunction(str(document))               # Removing special characters

            document = re.sub(r'\d+','',document)                                   # Removing numbers

            tokens = word_tokenize(document)                                        # Fetching Tokens

            tokens_lower = [word.lower() for word in tokens]                        # Removing stopwords   

            tokens_final = [word for word in tokens_lower if word not in stopWords and len(word) > 1]

            fileDetails[idOfDocument] = os.path.basename(filePath)

            fileData.append(tokens_final)
            idOfDocument += 1
            uq_dict = uniqueWordsFinderFunction(tokens_final)
            uniqueWordsDict.update(uq_dict)                                         # Storing unique words along with their frequencies
    unique_words = list(set(uniqueWordsDict))                                       # Creating list of unique words in whole dataset
    return uniqueWordsDict,fileDetails,fileData

In [40]:
# Calling pre-process function on the dataset
datasetPath="/content/drive/MyDrive/Assignments/IR Assignment 2/Data/Humor,Hist,Media,Food/*"
uniqueWordsDictFinal,fileDetailsFinal,fileDataFinal=preprocessingData(datasetPath)

File Id   File Name
1  	  grommet
2  	  rinaldos
3  	  hell
4  	  mowers
5  	  harmful
6  	  hate
7  	  herb!
8  	  mothers
9  	  howlong
10  	  roach
11  	  icm
12  	  murph
13  	  conan
14  	  hi
15  	  imprrisk
16  	  coyote
17  	  bible
18  	  dirtword
19  	  incarhel
20  	  horoscop
21  	  insanity
22  	  murphy
23  	  impurmat
24  	  interv
25  	  cartoon_
26  	  investi
27  	  bless
28  	  insure
29  	  terms
30  	  japrap
31  	  jac&tuu
32  	  ivan
33  	  killer
34  	  dym
35  	  test
36  	  blooprs1
37  	  kid_diet
38  	  lawsuniv
39  	  kilsmur
40  	  testchri
41  	  test2
42  	  killself
43  	  lbinter
44  	  motrbike
45  	  ludeinfo
46  	  simp
47  	  legal
48  	  murphys
49  	  lobquad
50  	  lll
51  	  office
52  	  lifeinfo
53  	  looser
54  	  lif&love
55  	  llong
56  	  lifeimag
57  	  livnware
58  	  losers86
59  	  ludeinfo
60  	  lozeuser
61  	  luggage
62  	  catstory
63  	  lozerzon
64  	  phunatdi
65  	  losers84
66  	  luzerzo2
67  	  manspace
68  	  manilla
69

In [42]:
#Creating function to calculate the IDF value
def evaluateIDF():
    documentFrequency = {}
    for i in fileDataFinal:
        documentTerms = list(set(i))
        for singleWord in documentTerms:
            if documentFrequency.get(singleWord) is not None:
                documentFrequency[singleWord] =documentFrequency.get(singleWord)+1
            else:
                documentFrequency[singleWord] = 1
    totalDocsValue=len(fileDataFinal)
    inverseDocumentFrequency = {}

    for word in documentFrequency.keys():
        inverse_val=np.log((totalDocsValue)/documentFrequency[word]+1)
        inverseDocumentFrequency[word] = inverse_val

    return documentFrequency,inverseDocumentFrequency

In [43]:
#Evaluating IDF values
documentFrequency,inverse_doc_freq=evaluateIDF()
print(documentFrequency)
print(inverse_doc_freq)

def evaluateTF():
    tfValuesList=[]
    for iterator in fileDataFinal:
        termFrequency={}
        for singleTerm in iterator:  
            if termFrequency.get(singleTerm) is not None:
                termFrequency[singleTerm] =termFrequency.get(singleTerm)+1
            else:
                termFrequency[singleTerm] = 1
        tfValuesList.append(termFrequency)

    return tfValuesList

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [44]:
#Creating function to evaluate TF-IDF value
termFrequency=evaluateTF()
def evaluateTFIDF(frequencyValue):
    tfIdfFinalList=[]
    for term in frequencyValue:
        tfIDFDict={}
        for word in term.keys():
            tfIDFDict[word]=term[word]*inverse_doc_freq[word]
        tfIdfFinalList.append(tfIDFDict)

    return tfIdfFinalList

In [45]:
#Defining function to evaluate binary TF
def evaluateBinaryTf(inputTermFrequency):
    binaryTFList= []
    for singleTerm in inputTermFrequency:
        tf = {}
        for singleWord in singleTerm.keys():
            if singleTerm.get(singleWord) < 0:
                tf[singleWord] = 0
            else:
                tf[singleWord] = 1
        binaryTFList.append(tf)
    return binaryTFList

In [149]:
#Calculating binary TF
binaryTF=evaluateBinaryTf(termFrequency)

In [150]:
#Evaluating TF-IDF value for the binary method
binaryTFIDF=evaluateTFIDF(binaryTF)
print(color.BOLD+"Binary TF-IDF value of the terms of the first document of the dataset:"+color.END)
print()
print("{0:22}{1}".format(color.BOLD+"Word","Binary TF-IDF Value"+color.END))
print()
for x,y in binaryTFIDF[0].items():
  #print(x," \t\t",y)
  print("{0:20}{1}".format(x, y))

Binary TF-IDF value of the terms of the first document of the dataset:

Word              Binary TF-IDF Value

help                1.56111275641069
grommets            5.936656310612987
missing             2.5306146475138815
happy               2.1117659956896504
fathers             3.2865344733420154
day                 1.259422260585345
eve                 3.4538411151475774
sipping             5.246146494219127
tasty               3.960813169597578
rheinhessen         6.341240749355558
back                1.123334681332636
porch               3.960813169597578
clattering          5.936656310612987
disturbed           3.960813169597578
oenophilic          6.341240749355558
reveries            5.936656310612987
looked              2.2465673803702275
eyes                1.9184297714726324
agog                6.341240749355558
stared              3.917226833303173
blankly             5.6498541326306455
rest                1.7976679059142213
family              1.867706099679588
lugged  

In [151]:
#Creating function to evaluate Raw TF value
def evaluateRawTFValue(inputTermFrequency):
    rawCountTF= inputTermFrequency
    return rawCountTF

rawCountTF=evaluateRawTFValue(termFrequency)

In [153]:
#Evaluating TF-IDF value for Raw Count Method
print(color.BOLD+"Raw TF-IDF value of the terms of the first document of the dataset:"+color.END)
print()
raw_count_tfidf=evaluateTFIDF(rawCountTF)
print("{0:24}{1}".format(color.BOLD+"Word","Raw TF-IDF Value"+color.END))
print()
for x,y in raw_count_tfidf[0].items():
  print("{0:20}{1}".format(x, y))

Raw TF-IDF value of the terms of the first document of the dataset:

Word                Raw TF-IDF Value

help                1.56111275641069
grommets            5.936656310612987
missing             2.5306146475138815
happy               4.223531991379301
fathers             9.859603420026046
day                 3.7782667817560345
eve                 3.4538411151475774
sipping             5.246146494219127
tasty               3.960813169597578
rheinhessen         6.341240749355558
back                3.3700040439979078
porch               7.921626339195156
clattering          5.936656310612987
disturbed           3.960813169597578
oenophilic          6.341240749355558
reveries            5.936656310612987
looked              2.2465673803702275
eyes                1.9184297714726324
agog                6.341240749355558
stared              3.917226833303173
blankly             5.6498541326306455
rest                1.7976679059142213
family              1.867706099679588
lugged      

In [50]:
def evaluateTermFrequency(inputTermFrequency):
    termFrequencyList = []
    for term in inputTermFrequency:
        tf = {}
        values = term.values() 
        t = sum(values) 
        for word in term.keys():
            tf[word]=term.get(word)/t
        termFrequencyList.append(tf)
    return termFrequencyList

In [154]:
#Calculating term frequency
termFrequencyFinal=evaluateTermFrequency(termFrequency)

In [155]:
print(color.BOLD+"TF-IDF value of the terms of the first document of the dataset:"+color.END)
print()
print("{0:24}{1}".format(color.BOLD+"Word","TF-IDF Value"+color.END))
print()
termFrequenctTfIdf=evaluateTFIDF(termFrequency)
for x,y in termFrequenctTfIdf[0].items():
  print("{0:20}{1}".format(x, y))

TF-IDF value of the terms of the first document of the dataset:

Word                TF-IDF Value

help                1.56111275641069
grommets            5.936656310612987
missing             2.5306146475138815
happy               4.223531991379301
fathers             9.859603420026046
day                 3.7782667817560345
eve                 3.4538411151475774
sipping             5.246146494219127
tasty               3.960813169597578
rheinhessen         6.341240749355558
back                3.3700040439979078
porch               7.921626339195156
clattering          5.936656310612987
disturbed           3.960813169597578
oenophilic          6.341240749355558
reveries            5.936656310612987
looked              2.2465673803702275
eyes                1.9184297714726324
agog                6.341240749355558
stared              3.917226833303173
blankly             5.6498541326306455
rest                1.7976679059142213
family              1.867706099679588
lugged              

In [156]:
#Creating function for evaluate log TF
def evaluatelogTF(inputTermFrequency):
    logTF = []
    for term in inputTermFrequency:
        tf = {}
        for word in term.keys():
            tf[word]=np.log(1+term.get(word))
        logTF.append(tf)
    return logTF

In [157]:
#Evaluating log TF
logTFFinal=evaluatelogTF(termFrequency)

In [158]:
#Evaluating log TF-IDF
logTFIDFFinal=evaluateTFIDF(logTFFinal)
print(color.BOLD+"Log TF-IDF value of the terms of the first document of the dataset:"+color.END)
print()
print("{0:24}{1}".format(color.BOLD+"Word","Log TF-IDF Value"+color.END))
print()
for x,y in logTFIDFFinal[0].items():
  print("{0:20}{1}".format(x, y))

Log TF-IDF value of the terms of the first document of the dataset:

Word                Log TF-IDF Value

help                1.0820809056422345
grommets            4.114976583654799
missing             1.7540884080079469
happy               2.3200120736560965
fathers             4.556104208020165
day                 1.745929978118329
eve                 2.394020231066561
sipping             3.6363516512724297
tasty               2.7454264812312617
rheinhessen         4.395413146667639
back                1.5572725343818423
porch               4.351398021238385
clattering          4.114976583654799
disturbed           2.7454264812312617
oenophilic          4.395413146667639
reveries            4.114976583654799
looked              1.5572018456415653
eyes                1.3297541871985152
agog                4.395413146667639
stared              2.7152147351178573
blankly             3.9161804626078873
rest                1.2460484405675436
family              1.2945952171075186
lugged

In [159]:
#Creating function for evaluating double log normalizes TF
def evaluateDoubleLogTF(inputTermFrequenct):
    doubleLogTFList = []
    for term in inputTermFrequenct:
        tf = {}
        tfMax=max(term.values())
        for word in term.keys():
            tf[word]=0.5 + 0.5*(term.get(word)/tfMax)
        doubleLogTFList.append(tf)
    return doubleLogTFList

In [160]:
#Calculating double log normalized term frequency
doubleLogFinal=evaluateDoubleLogTF(termFrequency)

In [161]:
doubleLogTFIDF=evaluateTFIDF(doubleLogFinal)
print(color.BOLD+"Double Log Normalised TF-IDF value of the terms of the first document of the dataset:"+color.END)
print()
print("{0:24}{1}".format(color.BOLD+"Word","Double Log TF-IDF Value"+color.END))
print()
for x,y in doubleLogTFIDF[0].items():
  print("{0:20}{1}".format(x, y))

Double Log Normalised TF-IDF value of the terms of the first document of the dataset:

Word                Double Log TF-IDF Value

help                0.9106491079062359
grommets            3.4630495145242426
missing             1.476191877716431
happy               1.4078439971264336
fathers             2.4649008550065115
day                 0.9445666954390086
eve                 2.0147406505027536
sipping             3.0602521216278245
tasty               2.310474348931921
rheinhessen         3.6990571037907425
back                0.8425010109994769
porch               2.640542113065052
clattering          3.4630495145242426
disturbed           2.310474348931921
oenophilic          3.6990571037907425
reveries            3.4630495145242426
looked              1.3104976385492995
eyes                1.1190840333590357
agog                3.6990571037907425
stared              2.2850489860935177
blankly             3.2957482440345434
rest                1.048639611783296
family         

In [59]:
#Creating and displaying the list of all available terms in document
allTermsList=list(documentFrequency.keys())
print(color.BOLD+"List of all terms present in document:"+color.END,allTermsList)
print(color.BOLD+"Toal number of terms present in document:"+color.END,len(allTermsList))

List of all terms present in document: ['provided', 'fine', 'gas', 'hubub', 'notsofond', 'dumped', 'one', 'kind', 'cant', 'blows', 'useless', 'spanish', 'manufacturer', 'piece', 'close', 'porch', 'send', 'set', 'ease', 'rest', 'looked', 'sunday', 'articles', 'gift', 'youll', 'procedure', 'witty', 'believes', 'manuals', 'defenestration', 'clattering', 'twoyearold', 'see', 'bike', 'lugged', 'grill', 'sturdily', 'tells', 'color', 'couple', 'falls', 'im', 'thoughts', 'effort', 'disturbed', 'venturi', 'newest', 'know', 'pleasure', 'drawing', 'freight', 'securing', 'household', 'spark', 'thats', 'continued', 'face', 'distracted', 'theory', 'huffed', 'english', 'hoop', 'assistance', 'ought', 'oenophilic', 'line', 'helper', 'way', 'husband', 'also', 'affix', 'japanese', 'cat', 'grommets', 'throwing', 'countless', 'windows', 'christmas', 'bring', 'product', 'mexican', 'packed', 'selfassembly', 'feet', 'long', 'help', 'american', 'ministrations', 'group', 'turned', 'repeat', 'missing', 'tortuous

In [60]:
#Creating function to find Top k documents using different TF-IDF versions
def calculate_top(tfIDFValues,inputQuery):
    tfIDFValuesList={}
    for iterator in range(len(fileDataFinal)+1):
        tfIDFValuesList[iterator] = 0.0
    for w in inputQuery: 
        docID=0
        for val in tfIDFValues:
            docID=docID+1
            if w in val:     
                tfIDFValuesList[docID]=tfIDFValuesList[docID]+val[w]
    sortedDict=dict(sorted(tfIDFValuesList.items(),key=operator.itemgetter(1),reverse=True)) 
    print(color.BLUE+""+str(sortedDict)+""+color.END)
    xc=0
    tfIDFFileList=[]
    for item in sortedDict.keys():
        if (xc != 5):
            tfIDFFileList.append(item)
        else: 
            break
        xc=xc+1   
    return tfIDFFileList

In [61]:
def process_query():
    query_input=input("Enter Query: ")
    print()
    query_preprocessed=removeSpecialCharactersFunction(query_input)
    tokens = word_tokenize(query_preprocessed )
    query_final = [word.lower() for word in tokens if word not in stopWords and len(word) > 1]
    print(color.BOLD+"Words in query after pre-processing:"+color.END,color.BLUE+"",query_final,""+color.END)
    print()
    print(color.BOLD+"Document ID and TD-IDF Score for all the documents given by Binary IF-IDF:"+color.END)
    binary=calculate_top(binaryTFIDF,query_final)
    print()
    print(color.BOLD+"Document ID and TD-IDF Score for all the documents given by Raw Count IF-IDF:"+color.END)
    raw_count=calculate_top(raw_count_tfidf,query_final)
    print()
    print(color.BOLD+"Document ID and TD-IDF Score for all the documents given by Term Frequency IF-IDF:"+color.END)
    termfreq=calculate_top(termFrequenctTfIdf,query_final)
    print()
    print(color.BOLD+"Document ID and TD-IDF Score for all the documents given by Log IF-IDF:"+color.END)
    log=calculate_top(logTFIDFFinal,query_final)
    print()
    print(color.BOLD+"Document ID and TD-IDF Score for all the documents given by Double Log IF-IDF:"+color.END)
    doublelog=calculate_top(doubleLogTFIDF,query_final)
    print()
    return binary,raw_count,termfreq,log,doublelog

In [62]:
bValue,rValue,tfValue,lValue,dlValue=process_query()
print()
print(color.BOLD+"Top 5 Documents according to Binary TF-IDF:"+color.END)
for i in bValue:
    print(color.BLUE+""+fileDetailsFinal[i]+""+color.END)

print(color.BOLD+"Top 5 Documents according to Raw Count TF-IDF:"+color.END)
for i in rValue:
    print(color.BLUE+""+fileDetailsFinal[i]+""+color.END)

print(color.BOLD+"Top 5 Documents according to TermFreq TF-IDF:"+color.END)
for i in tfValue:
    print(color.BLUE+""+fileDetailsFinal[i]+""+color.END)

print(color.BOLD+"Top 5 Documents according to Log TF-IDF:"+color.END)
for i in lValue:
    print(color.BLUE+""+fileDetailsFinal[i]+""+color.END)

print(color.BOLD+"Top 5 Documents according to Double Log TF-IDF:"+color.END)
for i in dlValue:
    print(color.BLUE+""+fileDetailsFinal[i]+""+color.END)

Enter Query: lion

Words in query after pre-processing:  ['lion'] 

Document ID and TD-IDF Score for all the documents given by Binary IF-IDF:
{48: 4.006467009344622, 55: 4.006467009344622, 115: 4.006467009344622, 116: 4.006467009344622, 245: 4.006467009344622, 330: 4.006467009344622, 382: 4.006467009344622, 508: 4.006467009344622, 509: 4.006467009344622, 511: 4.006467009344622, 554: 4.006467009344622, 601: 4.006467009344622, 625: 4.006467009344622, 638: 4.006467009344622, 693: 4.006467009344622, 756: 4.006467009344622, 799: 4.006467009344622, 884: 4.006467009344622, 942: 4.006467009344622, 959: 4.006467009344622, 1108: 4.006467009344622, 0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0, 14: 0.0, 15: 0.0, 16: 0.0, 17: 0.0, 18: 0.0, 19: 0.0, 20: 0.0, 21: 0.0, 22: 0.0, 23: 0.0, 24: 0.0, 25: 0.0, 26: 0.0, 27: 0.0, 28: 0.0, 29: 0.0, 30: 0.0, 31: 0.0, 32: 0.0, 33: 0.0, 34: 0.0, 35: 0.0, 36: 0.0, 37: 0.0, 38: 0.0, 39: 0.0, 40: